In [ ]:
from __future__ import print_function
import keras
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, BatchNormalization, Flatten
from keras.layers import Conv2D, MaxPooling2D
import os

In [ ]:
NUM_OF_CLASSES = 7
IMG_SIZE = 48
BATCH_SIZE = 128

TRAIN_DIR = 'data/train'
TEST_DIR = 'data/test'

CHECKPOINT_PATH = 'best_weights.ckpt'
EPOCHS = 25

In [ ]:
train_gen_obj = ImageDataGenerator(
    rescale=1. / 255,
    rotation_range=30,
    width_shift_range=0.4,
    height_shift_range=0.4,
    horizontal_flip=True,
    shear_range=0.3,
    fill_mode='nearest')

test_gen_obj = ImageDataGenerator(rescale=1. / 255)

In [ ]:
train_data = train_gen_obj.flow_from_directory(
    TRAIN_DIR,
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    shuffle=True,
    color_mode='grayscale')

test_data = test_gen_obj.flow_from_directory(
    TEST_DIR,
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    shuffle=True,
    color_mode='grayscale')


In [ ]:
from matplotlib import pyplot as plt
import matplotlib.image as mpimg

def shuffle_list(list_to_shuffle):
    import random
    random.shuffle(list_to_shuffle)
    return list_to_shuffle

def show_data_sample(sample_size:int, dir_path: str):
    plt.figure(figsize=(6,6))

    images = shuffle_list(os.listdir(dir_path))[:sample_size]
    for i in range(sample_size):
        file = mpimg.imread(f"{dir_path}/{images[i]}")
        plt.subplot(int(sample_size/2),2,i+1)
        plt.imshow(file)

show_data_sample(4, f'{TRAIN_DIR}/sad')
show_data_sample(4, f'{TRAIN_DIR}/surprised')
show_data_sample(4, f'{TRAIN_DIR}/happy')
show_data_sample(4, f'{TRAIN_DIR}/angry')
show_data_sample(4, f'{TRAIN_DIR}/fearful')
show_data_sample(4, f'{TRAIN_DIR}/disgusted')
show_data_sample(4, f'{TRAIN_DIR}/neutral')



In [ ]:
from keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau

checkpoint = ModelCheckpoint(
    filepath=CHECKPOINT_PATH, 
    verbose=1, 
    save_best_only=True, 
    monitor='val_loss',
    mode='max')

earlystopping = EarlyStopping(
    monitor='val_accuracy', 
    patience=5, 
    verbose=1,
    restore_best_weights=True)

reduce_lr = ReduceLROnPlateau(
    monitor='val_loss',
    factor=0.2,
    patience=2,
    verbose=1,
    min_lr=0.0000001,
    mode='auto')

callbacks = [checkpoint, earlystopping, reduce_lr]

In [ ]:
model = Sequential()

# Block-1

model.add(Conv2D(32,(3,3),padding='same',kernel_initializer='he_normal',input_shape=(IMG_SIZE,IMG_SIZE,1)))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(Conv2D(32,(3,3),padding='same',kernel_initializer='he_normal',input_shape=(IMG_SIZE,IMG_SIZE,1)))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.2))

# Block-2 

model.add(Conv2D(64,(3,3),padding='same',kernel_initializer='he_normal'))
model.add(Activation('relu'))
model.add(BatchNormalization())

model.add(Conv2D(64,(3,3),padding='same',kernel_initializer='he_normal'))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.2))

# Block-3

model.add(Conv2D(128,(3,3),padding='same',kernel_initializer='he_normal'))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(Conv2D(128,(3,3),padding='same',kernel_initializer='he_normal'))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.2))

# Block-4 

model.add(Conv2D(256,(3,3),padding='same',kernel_initializer='he_normal'))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(Conv2D(256,(3,3),padding='same',kernel_initializer='he_normal'))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.2))

# Block-5

model.add(Flatten())
model.add(Dense(64,kernel_initializer='he_normal'))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(Dropout(0.5))

# Block-6

model.add(Dense(64,kernel_initializer='he_normal', activation='relu'))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(Dropout(0.5))

# Block-7

model.add(Dense(NUM_OF_CLASSES,kernel_initializer='he_normal'))
model.add(Activation('softmax'))

In [ ]:
from keras.optimizers import adam_v2
model.compile(
    loss='categorical_crossentropy', 
    optimizer=adam_v2.Adam(learning_rate=0.001), 
    metrics=['accuracy'])

In [ ]:
import PIL
history = model.fit(
    train_data, 
    epochs=EPOCHS,
    callbacks = callbacks,
    validation_data = test_data,
    validation_steps = test_data.samples // BATCH_SIZE,
    steps_per_epoch = train_data.samples // BATCH_SIZE)